# Pre-processing

Loading of the graph generator

In [0]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
import subprocess
from time import time
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DataType, FloatType, MapType, Row
import pyspark.sql.functions as F 

In [0]:
dbutils.widgets.text("username", "", "Please enter your git username")
dbutils.widgets.text("password", "", "Please enter your password")

In [0]:
time.sleep(5)

In [0]:
# username = dbutils.widgets.get("username")
# password = dbutils.widgets.get("password")
username = 'sparkistan'
password = 'Nutella336kJ'
url = "https://" + username + ":" + password + "@github.com/MaximeRedstone/graphsMapReduce.git"
email = username + username + "@gmail.com"
dbutils.widgets.removeAll()

In [0]:
!git config --global user.email $email
!git config --global user.name $username
  
if os.path.exists(os.path.join(os.getcwd(), "graphsMapReduce")):
  !cd graphsMapReduce && git pull
else:
  !git clone $url

!cd graphsMapReduce/generator && make re
!mkdir graphsMapReduce/data

Already up to date.
make re graph_generator
make clean graph_generator
rm -f .o files
make fclean graph_generator
rm -f .o files
rm -f libft.a
rm -f ft_printf
make libft.a
ar: creating libft.a
make graph_generator
mkdir: cannot create directory ‘graphsMapReduce/data’: File exists

In [0]:
user_inputs = {
  "10M": 5,
  "30M": 5,
  "60M": 5
}

filestore_paths_dict = {}
for k, v in user_inputs.items():
  
  filename = "graph_" + k + ".txt"
  rel_filepath = os.path.join("..", "data", filename)
  abs_filepath = os.path.join("graphsMapReduce", "data",filename)
  !cd graphsMapReduce/generator && ./graph_generator $k $rel_filepath $v
  
  filestore_name = "dbfs:/FileStore/" + filename
  dbutils.fs.rm(filestore_name)
  f = open(abs_filepath, "r")
  string = f.read()
  dbutils.fs.put(filestore_name, string)
  filestore_paths_dict[(k,v)] = filestore_name
  
filename = "toy_dataset.txt"
rel_filepath = os.path.join("..", "data", filename)
abs_filepath = os.path.join("graphsMapReduce", "data",filename)
filestore_name = "dbfs:/FileStore/" + filename
f = open(abs_filepath, "r")
string = f.read()
try:
  dbutils.fs.put(filestore_name, string)
except:
  pass
filestore_paths_dict['toy'] = filestore_name

Start program at Thu Mar 18 14:52:57 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 166950
Final number of edges approximation is 834754
End program at Thu Mar 18 14:52:58 2021
Wrote 10485767 bytes.
Start program at Thu Mar 18 14:53:05 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 466559
Final number of edges approximation is 2332796
End program at Thu Mar 18 14:53:06 2021
Wrote 31457282 bytes.
Start program at Thu Mar 18 14:53:09 2021
Has parsed user argument
Has converted user argument to bytes
Final number of nodes is 914619
Final number of edges approximation is 4573098
End program at Thu Mar 18 14:53:13 2021
Wrote 62914565 bytes.

# RDD implementation

In [0]:
def processGraphRDD(filepath):
  
  def reduce_ccf(x):
    key = x[0]
    values = x[1]
    min_value = values.pop(values.index(min(values)))
    ret = []
    if min_value < key:
      ret.append((key, min_value))
      for value in values:
        accum.add(1) # là on peu surement add en une fois la taille de la liste
        ret.append((value, min_value)) # là on peut surement zip 
    return (ret)
  
  
  text_file = sc.textFile(filepath)
  
  text_file = text_file.filter(lambda x: "#" not in x)
  text_file_split = text_file.map(lambda x: x.split())
  input = text_file_split.map(lambda x: (int(x[0]), int(x[1])))

  accum = sc.accumulator(1)
  while accum.value > 0:

    accum.value = 0
    print("Start loop ", accum.value)

    # CCF-Iterate
    it_map = input.flatMap(lambda x: ((x[0], x[1]), (x[1], x[0])))
#     it_groupby = it_map.groupByKey().mapValues(lambda x: sorted(x))
    it_groupby = it_map.groupByKey().mapValues(list)
    it_reduce = it_groupby.flatMap(lambda x: reduce_ccf(x))
    
    # CCF-Dedup
    ded_map = it_reduce.map(lambda x: ((x[0], x[1]), None))
    ded_groupby = ded_map.groupByKey().mapValues(list)
    input = ded_groupby.map(lambda x: (x[0][0], x[0][1]))

    viz = input.collect()
    print('-------------')

  print("Processed file")
#   print(viz)

In [0]:
processGraphRDD(filepath)

In [0]:
!ls graphsMapReduce/data

In [0]:
event_dir = str(subprocess.check_output(['ls', 'eventlogs']))
event_dir = int(event_dir.split("\'")[1][:-2])
full_old_path = os.path.join('eventlogs', str(event_dir), 'eventlog')
full_new_path = os.path.join('eventlogs', str(event_dir), 'eventlog.json')

In [0]:
!cd graphsMapReduce && git pull

In [0]:
comment = ""
!pip install logzero
!mv $full_old_path $full_new_path
!cd graphsMapReduce/src && python3 main.py -e $event_dir #-c $comment

In [0]:
!cd graphsMapReduce/generator && make fclean
!cd graphsMapReduce && git add -A
!cd graphsMapReduce && git config --global user.email "sparkistan@gmail.com"
!cd graphsMapReduce && git config --global user.name "sparkistan"
!cd graphsMapReduce && git commit -m "Added logs"
!cd graphsMapReduce && git push $url

# DataFrame implementation

In [0]:
## CURRENT WORKING VERSION ##

def graphsDF(filestore_name):
  
  def reduce_ccf(key, values):
    min_value = values.pop(values.index(min(values)))
    ret = {}
    ret[key] = min_value
    if min_value < key:
      for value in values:
        acc.add(1)
        ret[value] = min_value
        #ret.append((value, min_value)) # là on peut surement zip 
    else:
      ret = None
    return ret

  reducer = F.udf(lambda x, y: reduce_ccf(x, y), MapType(IntegerType(), IntegerType()))

  schema = StructType([
      StructField("key", IntegerType(), True),
      StructField("value", IntegerType(), True)])

  df = spark.read.format('csv').load(filestore_name, headers=False, delimiter='\t', schema=schema)
  df = df.na.drop()
  


  extracted_val = 1
  acc = sc.accumulator(0)

  while extracted_val != 0:
    previous = acc.value
    # CCF-Iterate
    df_inverter = df.select(F.col('value').alias('key'), F.col('key').alias('value'))
    df = df.union(df_inverter)

      # Creates the (key, [values]) pairs
    df = df.groupBy('key').agg(F.collect_list('value').alias('value'))

    #df = df.groupBy(F.map_keys('key')[0].alias('key')).agg(F.collect_list(F.map_values('key')[0]).alias('value'))
    acc.value = 0
    df = df.withColumn('reducer', reducer('key', 'value')).select('reducer')
    df.collect()
    current = acc.value
    #df.show()
    # Explode & clear nones
    df = df.select(F.explode('reducer'))
    df = df.na.drop()
  #   print('Final output: ')
  #   df.show()

    # CCF - Dedup
    df = df.dropDuplicates()
    #df.show()
    extracted_val = current - previous
    #extracted_val += 1
    print('Final extract :', extracted_val)


  return df

filestore_name = filestore_paths_dict[("5M", 5)]
df = graphsDF(filestore_name)
#df.show()

Final extract : 730604
Final extract : 1447403
Final extract : 2838460
Final extract : 2257661
Final extract : 60710
Final extract : 0

In [0]:
## CURRENT WORKING VERSION WITH LOGS##

def graphsDF(filestore_name):
  
  def reduce_ccf(key, values):
    min_value = values.pop(values.index(min(values)))
    ret = {}
    ret[key] = min_value
    if min_value < key:
      for value in values:
        acc.add(1)
        ret[value] = min_value
        #ret.append((value, min_value)) # là on peut surement zip 
    else:
      ret = None
    return ret

  reducer = F.udf(lambda x, y: reduce_ccf(x, y), MapType(IntegerType(), IntegerType()))

  schema = StructType([
      StructField("key", IntegerType(), True),
      StructField("value", IntegerType(), True)])

  df = spark.read.format('csv').load(filestore_name, headers=False, delimiter='\t', schema=schema)
  df = df.na.drop()

  extracted_val = 1
  acc = sc.accumulator(0)

  while extracted_val != 0:
    previous = acc.value
    # CCF-Iterate
    df_inverter = df.select(F.col('value').alias('key'), F.col('key').alias('value'))
    df = df.union(df_inverter)

      # Creates the (key, [values]) pairs
    df = df.groupBy('key').agg(F.collect_list('value').alias('value'))

    #df = df.groupBy(F.map_keys('key')[0].alias('key')).agg(F.collect_list(F.map_values('key')[0]).alias('value'))
    acc.value = 0
  #   print(acc.value)
    df = df.withColumn('reducer', reducer('key', 'value')).select('reducer')
    df.collect()
    current = acc.value
    #df.show()
    # Explode & clear nones
    df = df.select(F.explode('reducer'))
    df = df.na.drop()
  #   print('Final output: ')
  #   df.show()

    # CCF - Dedup
    df = df.dropDuplicates()
    #df.show()
    extracted_val = abs(current - previous)
    #extracted_val += 1
    print('Final extract :', extracted_val)

  return df

filestore_name = filestore_paths_dict[("10M", 5)]
df = graphsDF(filestore_name)
#df.show()

Final extract : 1355603
Final extract : 2683303
Final extract : 5305417
Final extract : 5251057
Final extract : 129530
Final extract : 0

In [0]:
## WIP ##

#def graphsDF():
  
def reduce_ccf(key, values):
  min_value = values.pop(values.index(min(values)))
  new_keys = []
  new_values = []
  new_keys.append(key)
  new_values.append(min_value)
  ret[key] = min_value
  if min_value < key:
    for value in values:
      acc.add(1)
      new_keys.append(value)
      new_values.append(min_value)
      #ret.append((value, min_value)) # là on peut surement zip 
  else:
    ret = None
  return [new_keys, new_values]

udf_schema = StructType([
  StructField("key", IntegerType()),
  StructField("value", IntegerType())])

reducer = F.udf(lambda x, y: reduce_ccf(x, y), udf_schema)

schema = StructType([
    StructField("key", IntegerType(), True),
    StructField("value", IntegerType(), True)])

df = spark.read.format('csv').load(filestore_name, headers=False, delimiter='\t', schema=schema)

extracted_val = 1
acc = sc.accumulator(0)

while extracted_val != 0:
  
  # CCF-Iterate
  df_inverter = df.select(F.col('value').alias('key'), F.col('key').alias('value'))
  df = df.union(df_inverter)

    # Creates the (key, [values]) pairs
  df = df.groupBy('key').agg(F.collect_list('value').alias('value'))

  #df = df.groupBy(F.map_keys('key')[0].alias('key')).agg(F.collect_list(F.map_values('key')[0]).alias('value'))
  acc.value = 0
  df = df.withColumn('reducer', reducer('key', 'value')).select('reducer')
  df.collect()
  extracted_val = acc.value
  #df.show()
  # Explode & clear nones
  df = df.select(F.explode('reducer'))
  df = df.na.drop()
#   print('Final output: ')
#   df.show()

  # CCF - Dedup
  df = df.dropDuplicates()
  #df.show()
  print('Final extract :', extracted_val)
  #del acc

#  return df

#df = graphsDF()
df.show()

# Logs comparison

In [0]:
def add_log(df, filepath, algo_type, loop_counter, command, timestamp, accum):
    
    single_row = pd.DataFrame(columns=['filename', 'algo_type', 'loop_counter', 'command', 'end_of_command', 'accum'])
    single_row.loc[0] = [filepath, algo_type, loop_counter, command, timestamp, accum]
    with_row = pd.concat([df, single_row])
    return with_row

def processGraphRDDLogs(filepath, df_logs):
  
    def reduce_ccf_min(x):
        key = x[0]
        values = x[1]
        min_value = values.pop(np.argmin(values))
        ret = []
        if min_value < key:
            ret.append((key, min_value))
            accum.add(len(values))
            for value in values:
                ret.append((value, min_value))
        return (ret)

    text_file = sc.textFile(filepath)
    text_file = text_file.filter(lambda x: "#" not in x)
    
    df_logs = add_log(df_logs, filepath, "rdd", 0, "start", datetime.now(), 0)
    text_file_split = text_file.map(lambda x: x.split())
    input = text_file_split.map(lambda x: (int(x[0]), int(x[1])))

    accum = sc.accumulator(1)
    loop_counter = 1
    while accum.value > 0:

        accum.value = 0
        print(f"----------\nStart loop at {datetime.now()}", accum.value)

        # CCF-Iterate
        it_map = input.flatMap(lambda x: ((x[0], x[1]), (x[1], x[0])))
        it_map.collect()
        df_logs = add_log(df_logs, filepath, "rdd", loop_counter, "it_map", datetime.now(), accum.value)
        
        it_groupby = it_map.groupByKey().mapValues(list)
        it_groupby.collect()
        df_logs = add_log(df_logs, filepath, "rdd", loop_counter, "it_groupby", datetime.now(), accum.value)
        
        it_reduce = it_groupby.flatMap(lambda x: reduce_ccf_min(x))
        it_reduce.collect()
        df_logs = add_log(df_logs, filepath, "rdd", loop_counter, "it_reduce", datetime.now(), accum.value)

        # CCF-Dedup
        ded_map = it_reduce.map(lambda x: ((x[0], x[1]), None))
        ded_map.collect()
        df_logs = add_log(df_logs, filepath, "rdd", loop_counter, "ded_map", datetime.now(), accum.value)
        
        ded_groupby = ded_map.groupByKey().mapValues(list)
        ded_groupby.collect()
        df_logs = add_log(df_logs, filepath, "rdd", loop_counter, "ded_groupby", datetime.now(), accum.value)
        
        input = ded_groupby.map(lambda x: (x[0][0], x[0][1]))        
        viz = input.collect()
        df_logs = add_log(df_logs, filepath, "rdd", loop_counter, "ded_reduce", datetime.now(), accum.value)

        print(f"End loop at {datetime.now()}, accum_value is {accum.value}")
        loop_counter += 1

    print(f"----------\nProcessed file at {datetime.now()}\n----------")
    return df_logs
  
  
def processGraphDFLogs(filepath, df_logs):
  
  
  def reduce_ccf(key, values):
    min_value = values.pop(values.index(min(values)))
    ret = {}
    ret[key] = min_value
    if min_value < key:
      for value in values:
        acc.add(1)
        ret[value] = min_value
        #ret.append((value, min_value)) # là on peut surement zip 
    else:
      ret = None
    return ret

  reducer = F.udf(lambda x, y: reduce_ccf(x, y), MapType(IntegerType(), IntegerType()))

  schema = StructType([
      StructField("key", IntegerType(), True),
      StructField("value", IntegerType(), True)])

  df = spark.read.format('csv').load(filepath, headers=False, delimiter='\t', schema=schema)
  df = df.na.drop()
  df_logs = add_log(df_logs, filepath, "df", 0, "start", datetime.now(), 0)

  extracted_val = 1
  acc = sc.accumulator(0)
  loop_counter = 1
  
  while extracted_val != 0:
    previous = acc.value
    print(f"----------\nStart loop at {datetime.now()}, extracted val is {extracted_val}")
    
    # CCF-Iterate
    df_inverter = df.select(F.col('value').alias('key'), F.col('key').alias('value'))
    df = df.union(df_inverter)

      # Creates the (key, [values]) pairs
    df = df.groupBy('key').agg(F.collect_list('value').alias('value'))

    acc.value = 0
    df = df.withColumn('reducer', reducer('key', 'value')).select('reducer')
    df.collect()
    
    current = acc.value
    #df.show()
    # Explode & clear nones
    df = df.select(F.explode('reducer'))
    df = df.na.drop()
    
    df_logs = add_log(df_logs, filepath, "df", loop_counter, "it_reduce", datetime.now(), extracted_val)

    # CCF - Dedup
    df = df.dropDuplicates()
    #df.show()
    extracted_val = abs(current - previous)
    #df.collect()
    print(f"End loop at {datetime.now()}, final extract is {extracted_val}")
    loop_counter += 1
  
  print(f"----------\nProcessed file at {datetime.now()}\n----------")
  return df_logs

In [0]:
df_logs = pd.DataFrame(columns=['filename', 'data_structure', 'counter', 'command', 'end_of_command', 'accum'])
filestore_name = filestore_paths_dict[("30M", 5)]

In [0]:
df_logs_rdd = processGraphRDDLogs(filestore_name, df_logs)

----------
Start loop at 2021-03-18 17:46:14.548018 0
End loop at 2021-03-18 17:52:05.409622, accum_value is 11441019
----------
Start loop at 2021-03-18 17:52:05.409696 0
End loop at 2021-03-18 18:03:18.902712, accum_value is 22635846
----------
Start loop at 2021-03-18 18:03:18.903069 0
End loop at 2021-03-18 18:20:24.884311, accum_value is 44923488
----------
Start loop at 2021-03-18 18:20:24.884686 0
End loop at 2021-03-18 18:30:50.642875, accum_value is 57974673
----------
Start loop at 2021-03-18 18:30:50.643288 0
End loop at 2021-03-18 18:32:01.392233, accum_value is 2250294
----------
Start loop at 2021-03-18 18:32:01.392653 0
End loop at 2021-03-18 18:32:52.047883, accum_value is 0
----------
Processed file at 2021-03-18 18:32:52.047953
----------

In [0]:
df_logs_df = processGraphDFLogs(filestore_name, df_logs_rdd)

----------
Start loop at 2021-03-18 18:32:52.715481, extracted val is 1
End loop at 2021-03-18 18:33:20.280165, final extract is 3813673
----------
Start loop at 2021-03-18 18:33:20.280247, extracted val is 3813673
End loop at 2021-03-18 18:34:21.232932, final extract is 7545282
----------
Start loop at 2021-03-18 18:34:21.233349, extracted val is 7545282
End loop at 2021-03-18 18:36:17.308765, final extract is 14974496
----------
Start loop at 2021-03-18 18:36:17.309060, extracted val is 14974496
End loop at 2021-03-18 18:39:24.817702, final extract is 19324891
----------
Start loop at 2021-03-18 18:39:24.818086, extracted val is 19324891
End loop at 2021-03-18 18:42:56.153075, final extract is 750098
----------
Start loop at 2021-03-18 18:42:56.153250, extracted val is 750098
End loop at 2021-03-18 18:46:53.934190, final extract is 0
----------
Processed file at 2021-03-18 18:46:53.934282
----------

In [0]:
display(df_logs_df)

filename,data_structure,counter,command,end_of_command,accum,algo_type,loop_counter
dbfs:/FileStore/graph_30M.txt,NaN,NaN,start,2021-03-18T17:46:14.526+0000,0,rdd,0
dbfs:/FileStore/graph_30M.txt,NaN,NaN,it_map,2021-03-18T17:46:26.951+0000,0,rdd,1
dbfs:/FileStore/graph_30M.txt,NaN,NaN,it_groupby,2021-03-18T17:46:53.970+0000,0,rdd,1
dbfs:/FileStore/graph_30M.txt,NaN,NaN,it_reduce,2021-03-18T17:47:06.962+0000,3813673,rdd,1
dbfs:/FileStore/graph_30M.txt,NaN,NaN,ded_map,2021-03-18T17:47:24.657+0000,7627346,rdd,1
dbfs:/FileStore/graph_30M.txt,NaN,NaN,ded_groupby,2021-03-18T17:50:46.808+0000,11441019,rdd,1
dbfs:/FileStore/graph_30M.txt,NaN,NaN,ded_reduce,2021-03-18T17:52:05.397+0000,11441019,rdd,1
dbfs:/FileStore/graph_30M.txt,NaN,NaN,it_map,2021-03-18T17:53:29.929+0000,0,rdd,2
dbfs:/FileStore/graph_30M.txt,NaN,NaN,it_groupby,2021-03-18T17:55:17.777+0000,0,rdd,2
dbfs:/FileStore/graph_30M.txt,NaN,NaN,it_reduce,2021-03-18T17:55:34.069+0000,7545282,rdd,2
